# ZODB (Zope Object Database)

ZODB – объектно-ориентированная база данных, предназначенная для хранения Python-объектов. Написана на Python и является кроссплатформерной. Разработана Джимом Фултоном из Zope Corporation в конце 1990-х годов.

В ZODB используется прозрачная система хранения Python-объектов. Будучи объектно-ориентированной, она позволяет строить произвольные модели данных для конкретного приложения. Одним из основных достоинств является незаметность работы БД для пользователя - он просто выполняет обращения к объектам, ZODB делает всё остальное самостоятельно. ZODB полностью поддерживает модель ACID, что позволяет поддерживать целостность данных.

## Преимущества и недостатки

### Преимущества:
+ Прозрачная структура. Работа с данными идёт через прямое обращение к объектам;
+ Масштабируемая архитектура;
+ Не требуется никаких дополнительных языков для взаимодействия;
+ Использование базы данных практически не отличается от обычного обращения к объектам.

В ZODB имеется возможность использовать много разных архитектур хранения информации, подключаемых извне, вот некоторые из них:
+ Файловое хранилище для объектов на диске;
+ Каталогизированное хранилище. Объекты хранятся на диске, каждый в отдельном файле файловой системы;
+ Сетевое хранилище;
+ Хранилище RelStorage. Объекты сохраняются в реляционной СУБД;
+ Демо-хранилище. Используется для тестирования, доступно только для чтения.

### Недостатки:
+ Отсутствие встроенных механизмов сложных индексов. В ZODB нет таких мощных механизмов создания индексов, как в реляционных СУБД. Здесь используется перебор словаря для нахождения нужных значений. Это накладывает серьёзные ограничения на реализацию поиска по сложным критериям;
+ Отсутствие удобного доступа к данным из приложений реализованных не на Python. В случае реализации какого-либо сложного проекта на базе ZODB, придётся реализовывать API для доступа к данным, хранящимся в ней, для тех программ, которые выполнены не на Python.

## Применение

ZOBD следует использовать в следующих случаях:
+ Небольшие приложения

ZODB проста в использовании, не требует знания отдельных языков программирования и значительных изменений в коде приложения. ZODB подходит, когда разработка приложения является более важной задачей, чем продумывание взаимодействия с БД. 

+ Использование сложных взаимосвязей и структур данных

ZODB является объектно-ориентированной БД, что позволяет создавать структуры и взаимосвязи любой сложности, используя язык Python. Это значительно проще, чем создавать реляционные БД, где сложная структура будет требовать множества дорогостоящих объединений.

+ Доступ к данным через атрибуты и методы

ZODB не использует каких-либо специальных языков запросов. Поиск по БД здесь происходит через индексы, основанные на B-деревьях, с использованием высокоуровневого API для поиска. Это решение является приемлемым, если доступ через атрибуты и методы является более приоритетным, чем прямой поиск по базе данных.

+ Чтение происходит намного чаще записи

ZODB активно использует кэширование. Если рабочий набор помещается в памяти, база данных предоставляет очень высокую производительность, практически не выполняя запросов к серверу.

## Создание базы данных

In [1]:
import ZODB, ZODB.FileStorage

storage = ZODB.FileStorage.FileStorage('students_test.fs')
db = ZODB.DB(storage)

При создании базы данных автоматически создается хранилище.

## Подключение к базе данных

In [2]:
connection = db.open()

## Доступ к объектам

После соединения можно получить доступ к объектам, проходя по графу объектов из корневого объекта. Корневой объект базы данных-это объект сопоставления, который содержит объекты верхнего уровня в базе данных.

In [3]:
root = connection.root()

## Добавление объектов

Примитивный вариант выглядит следующим образом:

In [4]:
root['students'] = ['Maxim', 'Alexey', 'Andrey', 'Polina', 'Olga']

In [5]:
root

{'students': ['Maxim', 'Alexey', 'Andrey', 'Polina', 'Olga']}

## Сохранение изменений

После внесения всех правок требуется все сохранить, чтобы не потерять сделанное:

In [6]:
import transaction
transaction.commit()

и закрыть соединение:

In [7]:
connection.close()

После перезагрузки:

In [1]:
import ZODB, ZODB.FileStorage

storage = ZODB.FileStorage.FileStorage('students_test.fs')
db = ZODB.DB(storage)
connection = db.open()
root = connection.root()
root

{'students': ['Maxim', 'Alexey', 'Andrey', 'Polina', 'Olga']}

Данный способ плох тем, что в случае, если нужно будет добавить еще студента (с помощью команды append) даже в случае фиксации транзакции изменение не применится, так как ZODB не может самостоятельно обнаружить изменения в изменяемых объектах:

In [1]:
import ZODB, ZODB.FileStorage
import transaction

storage = ZODB.FileStorage.FileStorage('students_test.fs')
db = ZODB.DB(storage)
connection = db.open()
root = connection.root()

root['students'].append('Kirill')
print(root)
transaction.commit()
connection.close()

{'students': ['Maxim', 'Alexey', 'Andrey', 'Polina', 'Olga', 'Kirill']}


In [1]:
import ZODB, ZODB.FileStorage

storage = ZODB.FileStorage.FileStorage('students_test.fs')
db = ZODB.DB(storage)
connection = db.open()
root = connection.root()
root

{'students': ['Maxim', 'Alexey', 'Andrey', 'Polina', 'Olga']}

Самый простой и понятный способ создавать изменяемые объекты - создать постоянный класс.

In [1]:
import persistent
import persistent.list

class Student(persistent.Persistent):
    def __init__(self):
        self.debts = persistent.list.PersistentList()
    
    def setName(self, name):
        self.name = name
    
    # добавить задолженность
    def addDebt(self, debt):
        self.debts.append(debt)

Две основные задачи базового класса (Persistent) -определить , когда к объекту был получен доступ и когда он был изменен. Проблема с некорректным сохранением результатов изменяемых объектов решается с помощью использования постоянных структур данных (PersistentList и PersistentMapping).

In [3]:
import transaction
students=[]
for name in ['Polina', 'Olga']:
    student = Student()
    student.setName(name)
    students.append(student)
    
for name in ['Maxim', 'Alexey', 'Andrey']:
    student = Student()
    student.setName(name)
    student.addDebt('OODB')
    students.append(student)
    
root['students']=students
transaction.commit()
connection.close()

In [2]:
import ZODB, ZODB.FileStorage

storage = ZODB.FileStorage.FileStorage('students_test.fs')
db = ZODB.DB(storage)
connection = db.open()
root = connection.root()
root

{'students': [<__main__.Student object at 0x0000018A4160B4A8 oid 0x1f in <Connection at 18a4162d048>>, <__main__.Student object at 0x0000018A4160B518 oid 0x20 in <Connection at 18a4162d048>>, <__main__.Student object at 0x0000018A4160B588 oid 0x21 in <Connection at 18a4162d048>>, <__main__.Student object at 0x0000018A4160B5F8 oid 0x22 in <Connection at 18a4162d048>>, <__main__.Student object at 0x0000018A4160B668 oid 0x23 in <Connection at 18a4162d048>>]}

In [3]:
for student in root['students']:
    print(student.name, student.debts)

Polina []
Olga []
Maxim ['OODB']
Alexey ['OODB']
Andrey ['OODB']


## Пример с добавлением объектов в БД

In [1]:
from ZODB import DB
from ZODB.FileStorage import FileStorage
import persistent
import persistent.list
import transaction
import sys

class Student(persistent.Persistent):
    def __init__(self):
        self.debts = persistent.list.PersistentList()
    
    def setName(self, name):
        self.name = name
    
    # добавить задолженность
    def addDebt(self, debt):
        self.debts.append(debt)

storage=FileStorage("students_test.fs")
db=DB(storage)

In [2]:
connection=db.open()
root=connection.root()

if not 'students' in root:
    root['students'] = {}
students=root['students']


def listStudents():
    if len(students)==0:
        print("Students not found.")
        return
    for student in students:
        print("Name:", student.name)
        print("Debts:", 'None' if not student.debts else student.debts)
        print("\n")
if __name__=="__main__":
    while 1:
        choice=input("Press 'l' to list students, 'a' to add "
                         "a student, or 'q' to quit:")
        choice=choice.lower()
        if choice=="l":
            listStudents()
        elif choice=="a":
            name=input("Employee name:")
            
            if name in students:
                print("There is already an student with this name.")
                return
            
            student = Student()
            student.setName(name)
            
            while 1:
                choice2 = input("Press 'd' to add a debt, or 's' to stop:")
                if choice2=="d":
                    name=input("Debt name:")
                    student.addDebt(debt)
                elif choice2=="s":
                    students[name] = student
                    root['students'] = students
                    transaction.commit()
                    print("Student %s added.".format(name))
                    break
                    
        elif choice=="q":
            break

    connection.close()

Press 'l' to list students, 'a' to add a student, or 'q' to quit:l
Name: Polina
Debts: None


Name: Olga
Debts: None


Name: Maxim
Debts: ['OODB']


Name: Alexey
Debts: ['OODB']


Name: Andrey
Debts: ['OODB']


Press 'l' to list students, 'a' to add a student, or 'q' to quit:s
Press 'l' to list students, 'a' to add a student, or 'q' to quit:q


# Ссылки на источники

1. http://www.zodb.org
2. https://ru.bmstu.wiki/ZODB_(Zope_Object_Database)